In [1]:
from dotenv import load_dotenv
load_dotenv()
import requests
import json
import os
from pprint import pprint
from utilities import *
import pandas as pd
import pygsheets
from pangres import upsert
from sqlalchemy import text, create_engine


db_username = os.environ.get('db_username')
db_host = os.environ.get('db_host')
db_port = os.environ.get('db_port')
db_password = os.environ.get('db_password')
db_name = os.environ.get('db_name')

api_key = os.environ.get('riot_api_key')
my_puuid = os.environ.get('my_puuid')
test_match = os.environ.get('test_match')
rank1_puuid = os.environ.get('rank1_puuid')
json_dir = "JSONs"

service_account = pygsheets.authorize(service_account_file='JSONs/spreadsheetautomator-455618-bd558fe6d57a.json')

with open(os.path.join(json_dir, "archetype_items.json"), "r") as file:
    archetype_items = json.load(file)

with open(os.path.join(json_dir, "champ_classes.json"), "r") as file:
    classes = json.load(file)

with open(os.path.join(json_dir, "champ_items.json"), "r") as file:
    champ_items = json.load(file)

with open(os.path.join(json_dir, "champ_positions.json"), "r") as file:
    positions = json.load(file)

with open(os.path.join(json_dir, "most_recent.json"), "r") as file:
    recent = json.load(file)

with open(os.path.join(json_dir, "item_to_id.json"), "r") as file:
    item_to_id = json.load(file)

with open(os.path.join(json_dir, "perk_to_id.json"), "r") as file:
    perk_to_id = json.load(file)

with open(os.path.join(json_dir, "sums_to_id.json"), "r") as file:
    sums_to_id = json.load(file)

item_to_id = {int(k): v for k, v in item_to_id.items()}
perk_to_id = {int(k): v for k, v in perk_to_id.items()}
sums_to_id = {int(k): v for k, v in sums_to_id.items()}

def create_db_connection_string(db_username, db_password, db_host, db_port, db_name):
    conn_url = "postgresql+psycopg2://"+db_username+":"+db_password+'@' + db_host + ':' + db_port + '/' + db_name
    return conn_url

conn_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)

db_engine = create_engine(conn_url, pool_recycle=3600)
connection = db_engine.connect()



In [2]:
parse_match_data('na1', test_match, my_puuid)

,match_id,game_start,game_end,kills,deaths,assists,gold_earned,gold_spent,champ_level,neutral_minions_killed,...,secondary_1,secondary_2,atakhan,baron,champion,dragon,grubs,inhibitor,tower,rift_herald
0,NA1_5239301930,2025-02-28 20:42:23,2025-02-28 21:18:04,18,12,3,19913,18800,18,46,...,Biscuit Delivery,Approach Velocity,0,0,42,3,6,1,8,1


In [4]:
get_metabreakers_from_match_id('na1', test_match)

""


In [4]:
# pd.set_option('display.max_rows', None)
ladder = get_high_elo_players()

In [2]:
find_meta_breakers()

429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 seconds...
429 Forbidden - Retrying in 10 sec

KeyError: 'atakhan'

In [4]:
get_anomalous_players()

NameError: name 'get_anomalous_players' is not defined